In [1]:
import pyvista as pv
import numpy as np
from typing import Any, Tuple
from importlib import reload
import multiprocessing
import ifcopenshell
import ifcopenshell.geom
import time
from functools import reduce
import open3d as o3d
import random
import glob
import os
import pandas as pd
import threading

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def random_color():
    """Generate a random RGB color."""
    return [random.random(), random.random(), random.random()]

def open3d_block_by_element(ifc_file): 
    # Configuration for ifcopenshell geometric settings
    settings = ifcopenshell.geom.settings()
    settings.set(settings.USE_WORLD_COORDS, True)
    settings.set(settings.APPLY_DEFAULT_MATERIALS, True)
    
    # Initialize the iterator for geometric representations
    iterator = ifcopenshell.geom.iterator(settings, ifc_file, multiprocessing.cpu_count())
    
    # Define entity types to include and exclude
    exclude_list = set(["IfcCovering", "IfcFurnishingElement", "IfcSpace", "IfcOpening", "IfcOpeningElement", "IfcRailing"])
    
    # Lists to store the resulting meshes and element information
    all_meshes = []
    element_information = {}
    
    # Variable to track the smallest dimension among all meshes
    temp_dimension = 5.0
    
    # Iterate over the geometric representations
    if iterator.initialize():
        index = 0
        while True:
            shape = iterator.get()
            
            # Check if the shape type is not excluded and represents an IfcBuildingElement
            if shape.type not in exclude_list and shape.product.is_a("IfcBuildingElement"):
                # Convert the shape's geometry to an Open3D mesh
                faces = np.array(shape.geometry.faces)
                verts = np.array(shape.geometry.verts).reshape(-1, 3)
                mesh = o3d.geometry.TriangleMesh(vertices=o3d.utility.Vector3dVector(verts),
                                                 triangles=o3d.utility.Vector3iVector(faces.reshape(-1, 3)))
                
                # Update the smallest dimension if necessary
                min_dimension = min(mesh.get_max_bound() - mesh.get_min_bound())
                if min_dimension >= 0.2 and temp_dimension > min_dimension:
                    temp_dimension = round(float(min_dimension), 2)
                
                # Store the mesh and its corresponding IFC GUID
                all_meshes.append(mesh)
                element_information[index] = shape.guid
                index += 1
                
            # Move to the next geometric representation
            if not iterator.next():
                break

    return all_meshes, element_information, temp_dimension


def create_uniform_grid(bounds, voxel_size):
    """Create a uniform grid within the given bounds."""
    x = np.arange(bounds[0], bounds[1] + voxel_size, voxel_size)
    y = np.arange(bounds[2], bounds[3] + voxel_size, voxel_size)
    z = np.arange(bounds[4], bounds[5] + voxel_size, voxel_size)
    return pv.StructuredGrid(*np.meshgrid(x, y, z))

def open3d_to_pyvista(point_cloud_o3d):
    """
    Convert an Open3D point cloud to a PyVista point cloud.

    Parameters:
    - point_cloud_o3d: The Open3D point cloud.

    Returns:
    - A PyVista `PolyData` object.
    """
    # Extract points from Open3D point cloud
    points = np.asarray(point_cloud_o3d.points)

    # Create a PyVista PolyData object
    point_cloud_pv = pv.PolyData(points)

    return point_cloud_pv

def process_point(pcd, mesh_guid_index, grid_bounds, voxel_size, dims, grid_attributes):
    j = int((pcd[0] - grid_bounds[0]) / voxel_size)
    i = int((pcd[1] - grid_bounds[2]) / voxel_size)
    k = int((pcd[2] - grid_bounds[4]) / voxel_size)
    
    if 0 <= i < dims[0]-1 and 0 <= j < dims[1]-1 and 0 <= k < dims[2]-1 and grid_attributes[i,j,k] == -1:
        return (i, j, k, mesh_guid_index)
    return None

def apply_point_result(result, grid_attributes):
    if result is not None:
        i, j, k, mesh_guid_index = result
        grid_attributes[i,j,k] = mesh_guid_index

def process_points_thread(args):
    points, mesh_guid_index, grid_bounds, voxel_size, dims, grid_attributes = args
    for point in points:
        result = process_point(point, mesh_guid_index, grid_bounds, voxel_size, dims, grid_attributes)
        if result is not None:
            i, j, k, mesh_guid_index = result
            grid_attributes[i, j, k] = mesh_guid_index

def multithread_process_points(pcd_list, grid_bounds, voxel_size, dims, grid_attributes):
    threads = []
    for i, pcd in enumerate(pcd_list):
        t = threading.Thread(target=process_points_thread, args=((pcd.points, i, grid_bounds, voxel_size, dims, grid_attributes),))
        threads.append(t)
        t.start()
    
    for t in threads:
        t.join()

def voxelize_space(bounds, pcd_list, voxel_size):
    """Create a 3d grid and check the intersections of the meshes with the grid."""
    grid = create_uniform_grid(bounds, voxel_size)
    
    # create an empty 3d array in the same dimensions of the grid
    dims = grid.dimensions
    grid_attributes = np.full((dims[0]-1, dims[1]-1, dims[2]-1), -1, dtype=int)

    grid_bounds = grid.bounds

    # Use the parallel_process_points function to process points in parallel
    multithread_process_points(pcd_list, grid_bounds, voxel_size, dims, grid_attributes)

    # assign empty array to the grid
    grid.cell_data['attributes'] = grid_attributes.flatten(order='F').astype(int)
    
    return grid

def get_sampling_points(mesh, voxel_size, points_per_unit_area=120):
    
    # Calculate the surface area of the mesh
    area = mesh.get_surface_area()

    # Calculate the number of points to sample
    N = int(area * points_per_unit_area*1/voxel_size)
        
    return N

def create_point_cloud(all_meshes, voxel_size):
    """Create a point cloud from a mesh."""

    pcd_list = []
    
    for mesh in all_meshes:
        # Get the number of points to sample based on the mesh size
        N = get_sampling_points(mesh, voxel_size)
        pcd = mesh.sample_points_uniformly(N)
        pcd_pv = open3d_to_pyvista(pcd)
        pcd_list.append(pcd_pv)  
    
    return pcd_list

def fetch_concrete_elements(ifc_file, grid, voxel_size,element_information):
    elements = ifc_file.by_type("IfcBuildingElement")
    concrete_names = ["STB", "concrete", "beton"]
    
    concrete_elements = set()

    for element in elements:
        # Check ObjectType and Name attributes
        if any(concrete_name.lower() in (element.ObjectType or "").lower() for concrete_name in concrete_names) or \
           any(concrete_name.lower() in (element.Name or "").lower() for concrete_name in concrete_names):
            concrete_elements.add(element.GlobalId)
        else:
            # Check property sets
            for rel in element.IsDefinedBy:
                if rel.is_a("IfcRelDefinesByProperties"):
                    property_set = rel.RelatingPropertyDefinition
                    if property_set.is_a("IfcPropertySet"):
                        for prop in property_set.HasProperties:
                            if prop.is_a("IfcPropertySingleValue") and \
                               any(concrete_name.lower() in str(prop.NominalValue.wrappedValue).lower() for concrete_name in concrete_names):
                                concrete_elements.add(element.GlobalId)
                                break  # Exit once a match is found in the property set

    count = 0
    for att in grid.cell_data['attributes']:
        #print(att)
        if element_information.get(att) in concrete_elements:
            count += 1
    concrete_volume = round(count * voxel_size**3, 3)
    
    return concrete_volume    



In [3]:
def process_ifc_file(file_name):
    ifc_file = ifcopenshell.open(file_name)
    print(file_name)

    ### CONVERSION FROM IFC TO MESH ###
    print(f'Converting {file_name}...')
    start_time = time.time()
    all_meshes, element_information, voxel_size = open3d_block_by_element(ifc_file)
    total_polygons = sum(len(mesh.triangles) for mesh in all_meshes)
    # combine all meshes into one mesh and find the bounds
    combined_mesh = reduce(lambda m1, m2: m1 + m2, all_meshes)
    # Compute the oriented bounding box (OBB)
    obb = combined_mesh.get_axis_aligned_bounding_box()
    
    # Extract the eight corner points of the OBB
    obb_points = np.asarray(obb.get_box_points())

    # Compute the axis-aligned bounding box of these eight points
    xmin, ymin, zmin = np.min(obb_points, axis=0)
    xmax, ymax, zmax = np.max(obb_points, axis=0)

    bounds = np.array([xmin, xmax, ymin, ymax, zmin, zmax])

    end_time = time.time()
    conversion_time = end_time - start_time

    ### POINT CLOUD CREATION ###
    print(f'Point cloud is being formed for {file_name}...')
    start_time = time.time()
    point_cloud_list  = create_point_cloud(all_meshes, voxel_size)

    #merge all the list of point clouds into one
    pcd_pv = pv.PolyData()
    pcd_pv.points = np.vstack(list(map(lambda x: x.points, point_cloud_list)))
    sampling_points = len(pcd_pv.points)
    end_time = time.time()
    point_cloud_processing = end_time - start_time

    ### RASTERIZATION ###
    print(f'Rasterizing {file_name}...')
    start_time = time.time()
    grid = voxelize_space(bounds, point_cloud_list, voxel_size)
    end_time = time.time()
    rasterization_time = end_time - start_time

    ### VISUALIZATION ###
    print(f'Visualizing {file_name}...')
    start_time = time.time()
    p = pv.Plotter()  
    # Extract the cells with non-empty attribute values
    non_empty_cells = grid.cell_data['attributes'] != -1  
        
    # Add the mesh to the plotter with the non-empty cells
    p.add_mesh(grid.extract_cells(non_empty_cells), opacity=0.5, show_edges=False,show_scalar_bar=False)
    p.add_bounding_box(line_width=2, color="blue")
    screenshot_filename = os.path.join("screenshots", os.path.basename(file_name) + ".png")
    p.screenshot(screenshot_filename)
    p.show()
    end_time = time.time()
    visualization_time = end_time - start_time
    print(f'Calculating concrete volume of {file_name}...')

    start_time = time.time()
    concrete_volume = fetch_concrete_elements(ifc_file, grid, voxel_size, element_information)
    end_time = time.time()
    concrete_calculation_time = end_time - start_time

    # Store the results in a dictionary
    results = {}
    results['File'] = file_name
    results['Conversion Time'] = conversion_time
    results['Total Polygons'] = total_polygons
    results['Voxel Size'] = voxel_size
    results['Total Points'] = sampling_points
    results['Point Cloud Creation Time'] = point_cloud_processing
    results['Rasterization Time'] = rasterization_time
    results['Visualization Time'] = visualization_time
    results['Concrete Volume'] = concrete_volume
    results['Concrete Calculation Time'] = concrete_calculation_time

    print(results)
    return results

In [4]:
#process_ifc_file("IFC Files/04.ifc")

In [5]:
def find_ifc_files(directory):
    """Find all IFC files in a given directory."""
    # Use os.path.join to ensure the path is constructed correctly for any OS
    search_path = os.path.join(directory, "*.ifc")
    return glob.glob(search_path)

directory_path = "IFC Files/"

ifc_files = find_ifc_files(directory_path)

# Process each IFC file and store the results in a list
all_results = [process_ifc_file(file_name) for file_name in ifc_files]


IFC Files\01_Test.ifc
Converting IFC Files\01_Test.ifc...
Point cloud is being formed for IFC Files\01_Test.ifc...
Rasterizing IFC Files\01_Test.ifc...
Visualizing IFC Files\01_Test.ifc...


Widget(value="<iframe src='http://localhost:50577/index.html?ui=P_0x13883808970_0&reconnect=auto' style='width…

Calculating concrete volume of IFC Files\01_Test.ifc...
{'File': 'IFC Files\\01_Test.ifc', 'Conversion Time': 0.09023261070251465, 'Total Polygons': 1056, 'Voxel Size': 0.2, 'Total Points': 71470, 'Point Cloud Creation Time': 0.016180992126464844, 'Rasterization Time': 0.4869234561920166, 'Visualization Time': 1.9901635646820068, 'Concrete Volume': 9.592, 'Concrete Calculation Time': 0.005448579788208008}
IFC Files\02_Duplex.ifc
Converting IFC Files\02_Duplex.ifc...
Point cloud is being formed for IFC Files\02_Duplex.ifc...
Rasterizing IFC Files\02_Duplex.ifc...
Visualizing IFC Files\02_Duplex.ifc...


Widget(value="<iframe src='http://localhost:50577/index.html?ui=P_0x1389137c130_1&reconnect=auto' style='width…

Calculating concrete volume of IFC Files\02_Duplex.ifc...
{'File': 'IFC Files\\02_Duplex.ifc', 'Conversion Time': 2.8130080699920654, 'Total Polygons': 9420, 'Voxel Size': 0.2, 'Total Points': 2158977, 'Point Cloud Creation Time': 0.3498399257659912, 'Rasterization Time': 13.490779399871826, 'Visualization Time': 0.4833974838256836, 'Concrete Volume': 55.6, 'Concrete Calculation Time': 0.4761621952056885}
IFC Files\03_Clinic.ifc
Converting IFC Files\03_Clinic.ifc...
Point cloud is being formed for IFC Files\03_Clinic.ifc...
Rasterizing IFC Files\03_Clinic.ifc...
Visualizing IFC Files\03_Clinic.ifc...


...

Calculating concrete volume of IFC Files\03_Clinic.ifc...
{'File': 'IFC Files\\03_Clinic.ifc', 'Conversion Time': 21.268558740615845, 'Total Polygons': 63182, 'Voxel Size': 0.27, 'Total Points': 12774131, 'Point Cloud Creation Time': 2.5482370853424072, 'Rasterization Time': 80.99862718582153, 'Visualization Time': 1.2913951873779297, 'Concrete Volume': 17.183, 'Concrete Calculation Time': 7.327913045883179}
IFC Files\04.ifc
Converting IFC Files\04.ifc...
Point cloud is being formed for IFC Files\04.ifc...
Rasterizing IFC Files\04.ifc...
Visualizing IFC Files\04.ifc...


Widget(value="<iframe src='http://localhost:50577/index.html?ui=P_0x138913972b0_3&reconnect=auto' style='width…

Calculating concrete volume of IFC Files\04.ifc...
{'File': 'IFC Files\\04.ifc', 'Conversion Time': 2.183194398880005, 'Total Polygons': 16440, 'Voxel Size': 0.25, 'Total Points': 21111368, 'Point Cloud Creation Time': 3.365001678466797, 'Rasterization Time': 125.56527256965637, 'Visualization Time': 2.6058993339538574, 'Concrete Volume': 7716.156, 'Concrete Calculation Time': 1.5559840202331543}
IFC Files\05.ifc
Converting IFC Files\05.ifc...
Point cloud is being formed for IFC Files\05.ifc...
Rasterizing IFC Files\05.ifc...
Visualizing IFC Files\05.ifc...


Widget(value="<iframe src='http://localhost:50577/index.html?ui=P_0x138a82da770_4&reconnect=auto' style='width…

Calculating concrete volume of IFC Files\05.ifc...
{'File': 'IFC Files\\05.ifc', 'Conversion Time': 3.9632163047790527, 'Total Polygons': 39244, 'Voxel Size': 0.2, 'Total Points': 32082430, 'Point Cloud Creation Time': 5.397493839263916, 'Rasterization Time': 209.91610264778137, 'Visualization Time': 5.045258045196533, 'Concrete Volume': 9905.848, 'Concrete Calculation Time': 2.3943870067596436}
IFC Files\06.ifc
Converting IFC Files\06.ifc...
Point cloud is being formed for IFC Files\06.ifc...
Rasterizing IFC Files\06.ifc...


In [ ]:
# Convert the list of results to a DataFrame
df = pd.DataFrame(all_results)

# Write the DataFrame to an Excel file
df.to_excel("performance_results_parallel.xlsx", index=False)